In [69]:
import os
import glob
import pandas as pd
import re
import string
import itertools
import nltk
from nltk.stem import PorterStemmer
ps =nltk.PorterStemmer()
from nltk.corpus import stopwords
stops = set(stopwords.words("english")) 
print(stops)
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
#from stemming.porter2 import stem


prefix_path = 'C:\\Users\\Kirthika\\Desktop\\PES\\AIR Proj\\AIR\\Dataset'
extension = 'csv'
os.chdir(prefix_path)
filenames = glob.glob('*.{}'.format(extension))
#filenames

{"you've", 'will', 'needn', 'll', 'that', 'him', 'a', 'above', "didn't", 'here', 'being', 've', "shan't", 'didn', 'theirs', 'before', 'because', 'on', 'each', 'mightn', 'it', 'so', 'with', 'then', "she's", 't', "should've", 'most', 'at', 'ourselves', 'this', 'be', 'now', "aren't", "wouldn't", 'below', 'having', 'more', 'these', 'in', 'hadn', 'itself', 'nor', 'did', 'haven', 'too', 'mustn', 'shouldn', 'down', 'they', 'an', "you'd", 'is', 'out', 'were', 'some', 'does', 'isn', 'd', 'when', 'until', 'after', 'we', 'into', 'she', "that'll", 'wasn', 'while', 'other', 'the', 'don', "needn't", "weren't", 'further', 'about', 'few', 'how', "haven't", "mightn't", 'ain', 'those', 'y', 'himself', 'up', 'myself', 'hasn', 'yours', 'can', "couldn't", 'won', 'are', 'same', 'during', 'whom', "wasn't", 'my', 'them', 'between', 'all', 'any', "it's", 'which', "doesn't", 'have', 'than', 'as', 'to', 'has', "hadn't", 'if', 's', "won't", 'only', "don't", 'ours', 'through', 'themselves', 'there', 'over', 'was',

In [53]:
def get_files(files):
    doc_to_words = {}
    doc_to_words_clean = {}
    for file in files:
        #text = ""
        complete_path = prefix_path + '\\' + str(file)
        try:
            df = pd.read_csv(complete_path,sep=',')
            df.Snippet = df.Snippet.astype(str)
            text = df.Snippet
            doc_to_words[file] = text.to_list()
        except:
            #print("File is Empty :" + complete_path)
            continue

        
    return doc_to_words
        
    

In [54]:
doc_to_words = get_files(filenames)
print(doc_to_words[ 'BBCNEWS.201704.csv'][0:80])
document_test = ""

['bleached severely and this year, the central part bleached severely. it depends whether we have more of these mass events occurring in the next two years and get the global warming projections,', "and, this year, the central third of the reef has bleached severely. it's not irreparable but it does depend on whether we have more of these mass bleaching events occurring in the next few years. looking at the global warming trajectories,", 'neighbouring ones. what is the solution? rising ocean temperatures is causing this. and with man-made climate change, and', 'and will continue, creating a north-south divide in ourflora and fauna. we found as you might expect that under climate change, the', 'signed the letter represent but on behalf of the majority of people in the uk. two thirds of british people are concerned about climate change and when you ask what it is that people are most worried about in relation to climate change, 80% of', 'and his critics, will agree. meanwhile hours befor

In [7]:
# def clean_text(text):
#     text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
#     text_rc = re.sub('[0-9]+', '', text_lc)
#     tokens = re.split('\W+', text_rc)    # tokenization
#     text = [word for word in tokens if word not in stops]  # remove stopwords and stemming
#     return text
# def stemm(lis):
#     l = [stemming.stem(word) for word in lis]
#     return l

In [57]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stops]  # remove stopwords and stemming
    return text

In [58]:
for key,val in doc_to_words.items():
        cl_txt = [clean_text(item) for item in val]
        #st_txt = [stemm(item) for item in cl_txt]
        doc_to_words[key] = cl_txt
      

        
        

In [59]:
def map_pos(wordList):
    i_file = {}
    for idx, word in enumerate(wordList):
        if word in i_file.keys():
            i_file[word].append(idx)
        else:
            i_file[word] = [idx]
    return i_file

In [60]:
def hash_pos(doc_to_words_list):
    l = []
    hash_file_word_pos = {}
    for file in doc_to_words_list.keys():
        list_word = []
        l = doc_to_words_list[file]
        for w in l:
            w = map_pos(w)
            list_word.append(w)
        hash_file_word_pos[file] = list_word
    return hash_file_word_pos

In [63]:
hash_file_words = hash_pos(doc_to_words)

In [64]:
def compute_inverted_dict(hash_file_words):
    inverted_dict = {}
    for filename in hash_file_words.keys():
        for row_number in range(0,len(hash_file_words[filename])):
                for words in hash_file_words[filename][row_number]:
                    if words not in inverted_dict.keys():
                        inverted_dict[words] = {}
                    if filename not in inverted_dict[words].keys():
                        inverted_dict[words][filename] = {}
                    inverted_dict[words][filename][row_number] = hash_file_words[filename][row_number][words]
    return inverted_dict

In [65]:
inverted_dict = compute_inverted_dict(hash_file_words)

In [67]:
print(inverted_dict['out'])

{'BBCNEWS.201809.csv': {61: [4]}, 'CNN.201007.csv': {33: [18]}, 'CNN.201807.csv': {48: [4]}, 'CNN.201905.csv': {43: [19]}, 'FOXNEWS.201201.csv': {62: [0, 6], 81: [3]}, 'FOXNEWS.201507.csv': {31: [18]}, 'FOXNEWS.201604.csv': {5: [1], 6: [1]}, 'FOXNEWS.201904.csv': {128: [15]}, 'FOXNEWS.201906.csv': {281: [20]}, 'FOXNEWS.201909.csv': {571: [19]}, 'MSNBC.201701.csv': {24: [2]}, 'MSNBC.201706.csv': {408: [12]}}


In [38]:
import pickle
import json
with open('inverted_index.pickle', 'wb') as handle:
    pickle.dump(inverted_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [68]:
print(hash_file_words['FOXNEWS.201211.csv'])

[{'activ': [0], 'link': [1], 'global': [2], 'warm': [3], 'climat': [4, 17], 'chang': [5, 18], 'hurrican': [6], 'sophist': [7], 'possibl': [8], 'dishonest': [9], 'want': [10, 14], 'fair': [11], 'balanc': [12], 'the': [13], 'tun': [15], 'oreilli': [16], 'expert': [19]}, {'welcom': [0], 'program': [1], 'discuss': [2], 'whether': [3, 7], 'global': [4], 'warm': [5], 'real': [6], 'hurrican': [8], 'sandi': [9], 'caus': [10], 'climat': [11], 'chang': [12], 'interest': [13], 'carbon': [14], 'tax': [15], 'think': [16, 18], 'happen': [17]}, {'prescript': [0], 'flu': [1], 'forget': [2], 'pick': [3], 'cliff': [4, 7], 'go': [5], 'climat': [6], 'al': [8], 'gore': [9], 'call': [10], 'carbon': [11], 'tax': [12]}, {'greg': [0, 8], 'read': [1], 'paragraph': [2], 'bob': [3, 10], 'want': [4], 'bring': [5, 11], 'scientif': [6], 'research': [7], 'tomorrow': [9], 'side': [12], 'show': [13], 'climat': [14, 17], 'chang': [15, 18], 'result': [16]}, {'natur': [0], 'factor': [1, 6], 'major': [2, 3], 'driver': [4],

In [30]:
print((doc_to_words['FOXNEWS.201202.csv']))

[['trillion', 'dollar', 'cut', 'save', 'john', 'great', 'true', 'would', 'know', 'gone', 'pile', 'paper', 'turn', 'want', 'spend', 'solv', 'global', 'warm'], ['well', 'republican', 'gener', 'dont', 'want', 'inhibit', 'oil', 'industri', 'democrat', 'gener', 'want', 'high', 'ga', 'price', 'folk', 'switch', 'mode', 'transport', 'embrac', 'altern', 'energi', 'part', 'global', 'warm', 'deal'], ['work', 'john', 'thank', 'adam', 'bellow', 'come', 'global', 'warm', 'trust', 'go', 'roast', 'drown', 'return', 'well', 'hear', 'someon', 'peopl', 'hate', 'say', 'global'], ['fill', 'feed', 'famili', 'fill', 'go', 'without', 'someth', 'american', 'decid', 'know', 'got', 'snowbal', 'chanc', 'hell', 'reelect', 'bob', 'peopl', 'dana', 'global', 'warm'], ['dont', 'want', 'rais', 'tax', 'overal', 'pollut', 'tax', 'would', 'like', 'simultan', 'cut', 'incom', 'tax', 'payrol', 'tax', 'way', 'al', 'gore', 'suggest', 'everi', 'time', 'he', 'talk', 'carbon', 'tax'], ['bob', 'upset', 'bush', 'administr', 'white'

In [48]:
with open('inverted_index.json') as f:
    data = json.load(f)

In [51]:
data["climat"]

{'BBCNEWS.201701.csv': {'2': [31],
  '4': [1, 14],
  '5': [12, 17],
  '6': [4],
  '9': [2],
  '11': [8],
  '14': [11, 24],
  '15': [12],
  '16': [3, 12],
  '18': [13, 22],
  '20': [15],
  '21': [18],
  '22': [4],
  '23': [19],
  '24': [19],
  '25': [25],
  '26': [18],
  '27': [6, 15, 23],
  '28': [21],
  '29': [14],
  '30': [4],
  '31': [9],
  '32': [18],
  '33': [19],
  '34': [23],
  '35': [40],
  '36': [24],
  '38': [23],
  '40': [11],
  '41': [10],
  '50': [15],
  '63': [18],
  '67': [5, 11, 18],
  '68': [22],
  '69': [17],
  '70': [6, 15],
  '71': [22],
  '72': [4, 14],
  '73': [6, 16],
  '74': [25],
  '75': [21],
  '77': [11, 16],
  '78': [8, 14],
  '81': [4, 11],
  '82': [1, 9],
  '83': [23],
  '93': [12, 33],
  '94': [30],
  '98': [6],
  '103': [6],
  '105': [3],
  '112': [3],
  '113': [2],
  '114': [0],
  '115': [4],
  '120': [8],
  '121': [2],
  '122': [11],
  '125': [2],
  '127': [8],
  '128': [7],
  '129': [8],
  '130': [4],
  '131': [5],
  '132': [7],
  '133': [6],
  '134':